In [ ]:
import os
import numpy as np
from scipy.io import loadmat
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, WhiteKernel
from sklearn.metrics import mean_squared_error, make_scorer
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pickle

In [ ]:
dataset_size = 2000
test_size = 0.3

## Load data from mat

In [ ]:
data = loadmat('../mat_files/orange_straight_train.mat')
y = data['y_train']
X = data['X_train']
X[:,0] = [-10*x for x in X[:,0]]
X, y = resample(X, y, n_samples=dataset_size, replace=False, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

## Normalize data

In [ ]:
scaler = StandardScaler()
y_train_norm =scaler.fit_transform(y_train)
y_test_norm = scaler.transform(y_test)

## GPR

In [ ]:
#kernel = RBF([0.00469, 0.0039], length_scale_bounds='fixed') + WhiteKernel(0.0663, noise_level_bounds='fixed') # 2000, 0.3, aniso
kernel = RBF([0.006, 0.003], length_scale_bounds='fixed') + WhiteKernel(0.008, noise_level_bounds='fixed') # 2000, 0.3
#kernel = RBF([0.01, 0.01]) + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel,
                               normalize_y=False,
                               n_restarts_optimizer=10,
                               random_state=42)
gpr.fit(X_train, y_train_norm)


In [ ]:
#with open('gpr_rbf.pickle', 'wb') as f:
#    pickle.dump({'gpr': gpr, 'scaler': scaler}, f)

In [ ]:
#kernel = Matern([0.00312, 0.00363], length_scale_bounds='fixed', nu=1.5) + WhiteKernel(0.0307, noise_level_bounds='fixed') # 2000, 0.3, aniso
kernel = Matern([0.02, 0.01], length_scale_bounds='fixed', nu=1.5) + WhiteKernel(0.01, noise_level_bounds='fixed') # 2000, 0.3, aniso
#kernel = Matern([0.01, 0.01], nu=1.5) + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel,
                               normalize_y=False,
                               n_restarts_optimizer=10,
                               random_state=42)
gpr.fit(X_train, y_train_norm)

In [ ]:
#kernel = Matern([0.00246, 0.00287], length_scale_bounds='fixed', nu=2.5) + WhiteKernel(0.0369, noise_level_bounds='fixed') # 2000, 0.3, aniso
kernel = Matern([0.025, 0.015], length_scale_bounds='fixed', nu=2.5) + WhiteKernel(0.01, noise_level_bounds='fixed') # 2000, 0.3, aniso
#kernel = Matern([0.01, 0.01], nu=2.5) + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel,
                               normalize_y=False,
                               n_restarts_optimizer=10,
                               random_state=42)
gpr.fit(X_train, y_train_norm)

In [ ]:
#print(gpr.get_params())
print(gpr.kernel_)

## Plot

In [ ]:
X1, X2 = np.meshgrid(np.linspace(np.min(X_train[:,0]),np.max(X_train[:,0]),100),np.linspace(np.min(X_train[:,1]),np.max(X_train[:,1]),100))
Y = np.zeros((100,100))
Y_upper = np.zeros((100,100))
Y_lower = np.zeros((100,100))
for i in range(100):
    for j in range(100):
        res, std = gpr.predict(np.array([X1[i,j], X2[i,j]]).reshape(1,-1), return_std=True)
        Y[i,j] = scaler.inverse_transform(res)
        Y_upper[i,j] = Y[i,j] + 2*np.sqrt(std) * scaler.scale_
        Y_lower[i,j] = Y[i,j] - 2*np.sqrt(std) * scaler.scale_
        
        
#X1, X2 = np.meshgrid(np.sort(X_test[:,0]), np.sort(X_test[:,1]))
#Y = np.zeros(X1.shape)
#Y_upper = np.zeros(X1.shape)
#Y_lower = np.zeros(X1.shape)
#for i in range(Y.shape[0]):
#    for j in range(Y.shape[1]):
#        res, std = gpr.predict(np.array([X1[i,j], X2[i,j]]).reshape(1,-1), return_std=True)
#        Y[i,j] = scaler.inverse_transform(res)
#        Y_upper[i,j] = Y[i,j] + 2*np.sqrt(std) * scaler.scale_
#        Y_lower[i,j] = Y[i,j] - 2*np.sqrt(std) * scaler.scale_

fig = go.Figure(data=[
    go.Surface(z=Y, x=X1, y=X2),
    go.Scatter3d(x=X_train[:,0].ravel(), y=X_train[:,1].ravel(), z=y_train.ravel(), mode='markers',
                 marker=dict(opacity=1, size=1, color='black')),
    go.Surface(z=Y_upper, x=X1, y=X2, showscale=False, opacity=0.9),
    go.Surface(z=Y_lower, x=X1, y=X2, showscale=False, opacity=0.9),
    ])
fig.update_layout(scene = dict(
                    xaxis_title='Depth [cm]',
                    yaxis_title='Linear Velocity [m/s]',
                    zaxis_title='Force [N]'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
fig.show()